# Classification
## Premier League 2018/2019 season W/D/L prediction program

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

%matplotlib inline

In [2]:
df = pd.read_csv(r'D:\git_space\data\Premier-League_18-19seasons_matches.csv')
df.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1533927600,Aug 10 2018 - 7:00pm,complete,74439,Manchester United,Leicester City,Andre Marriner,1,0.0,0.0,...,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester)
1,1533987000,Aug 11 2018 - 11:30am,complete,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.0,0.0,...,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne)
2,1533996000,Aug 11 2018 - 2:00pm,complete,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.0,0.0,...,2.03,3.51,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset)
3,1533996000,Aug 11 2018 - 2:00pm,complete,24821,Fulham,Crystal Palace,Mike Dean,1,0.0,0.0,...,2.31,3.44,3.26,1.28,1.87,3.20,7.00,1.71,2.00,Craven Cottage (London)
4,1533996000,Aug 11 2018 - 2:00pm,complete,24121,Huddersfield Town,Chelsea,Chris Kavanagh,1,0.0,0.0,...,7.47,4.27,1.51,1.29,1.91,3.30,6.95,2.00,1.71,John Smith's Stadium (Huddersfield- West Yorks...


In [3]:
df.columns

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 64 columns):
timestamp                              380 non-null int64
date_GMT                               380 non-null object
status                                 380 non-null object
attendance                             380 non-null int64
home_team_name                         380 non-null object
away_team_name                         380 non-null object
referee                                380 non-null object
Game Week                              380 non-null int64
Pre-Match PPG (Home)                   380 non-null float64
Pre-Match PPG (Away)                   380 non-null float64
home_ppg                               380 non-null float64
away_ppg                               380 non-null float64
home_team_goal_count                   380 non-null int64
away_team_goal_count                   380 non-null int64
total_goal_count                       380 non-null int64
total_goals_

In [5]:
df.isnull().sum()

timestamp         0
date_GMT          0
status            0
attendance        0
home_team_name    0
                 ..
odds_ft_over35    0
odds_ft_over45    0
odds_btts_yes     0
odds_btts_no      0
stadium_name      0
Length: 64, dtype: int64

In [6]:
# 의미없는 column 삭제
meaningless_data = ['timestamp', 'date_GMT', 'status', 'attendance', 'referee', 'Game Week', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)',                  'average_goals_per_match_pre_match', 'btts_percentage_pre_match', 'over_15_percentage_pre_match', 'over_25_percentage_pre_match', 'over_35_percentage_pre_match', 'over_45_percentage_pre_match', 'over_15_HT_FHG_percentage_pre_match', 'over_05_HT_FHG_percentage_pre_match', 'over_15_2HG_percentage_pre_match', 'over_05_2HG_percentage_pre_match', 'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match', 'odds_ft_over15', 'odds_ft_over25', 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes', 'odds_btts_no', 'stadium_name']

# 경기시작 전에 알 수 없는 column 삭제(예측을 해야 하기 때문에 경기 기록 data를 사용하면 안된다)
unknown_data = ['total_goal_count', 'total_goals_at_half_time', 'home_team_goal_count_half_time', 'away_team_goal_count_half_time', 'home_team_goal_timings', 'away_team_goal_timings', 'home_team_corner_count', 'away_team_corner_count', 'home_team_yellow_cards', 'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards', 'home_team_first_half_cards', 'home_team_second_half_cards', 'away_team_first_half_cards', 'away_team_second_half_cards', 'home_team_shots', 'away_team_shots', 'home_team_shots_on_target', 'away_team_shots_on_target', 'home_team_shots_off_target', 'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls', 'home_team_possession', 'away_team_possession',]

df = df.drop(meaningless_data + unknown_data, axis=1)

df.head()

,home_team_name,away_team_name,home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win
0,Manchester United,Leicester City,1.89,1.32,2,1,1.25,1.46,1.37,4.98,9.81
1,Newcastle United,Tottenham Hotspur,1.32,1.74,1,2,1.18,1.58,4.51,3.77,1.84
2,AFC Bournemouth,Cardiff City,1.53,0.74,2,0,1.50,0.89,2.03,3.51,3.96
3,Fulham,Crystal Palace,1.11,1.53,0,2,1.67,1.81,2.31,3.44,3.26
4,Huddersfield Town,Chelsea,0.47,1.58,0,3,0.90,1.17,7.47,4.27,1.51


In [7]:
# 홈팀 득점 수 - 어웨이팀 득점 수 column 생성
df['difference'] = df['home_team_goal_count'] - df['away_team_goal_count']
df.head()

# difference column의 data가 양수(홈팀 승)면 2반환, 0(무승부)이면 1반환, 음수(홈팀 패)면 0반환
def func(x):
    if x > 0:
        return 2
    elif x == 0:
        return 1
    else:
        return 0

# 홈팀의 승무패 정보를 담는 column 생성
df['home_team_result'] = df['difference'].apply(lambda x : func(x))
df

,home_team_name,away_team_name,home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,difference,home_team_result
0,Manchester United,Leicester City,1.89,1.32,2,1,1.25,1.46,1.37,4.98,9.81,1,2
1,Newcastle United,Tottenham Hotspur,1.32,1.74,1,2,1.18,1.58,4.51,3.77,1.84,-1,0
2,AFC Bournemouth,Cardiff City,1.53,0.74,2,0,1.50,0.89,2.03,3.51,3.96,2,2
3,Fulham,Crystal Palace,1.11,1.53,0,2,1.67,1.81,2.31,3.44,3.26,-2,0
4,Huddersfield Town,Chelsea,0.47,1.58,0,3,0.90,1.17,7.47,4.27,1.51,-3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Liverpool,Wolverhampton Wanderers,2.79,1.21,2,0,1.40,0.92,1.32,5.80,9.00,2,2
376,Manchester United,Cardiff City,1.89,0.74,0,2,2.67,1.31,1.28,6.20,10.00,-2,0
377,Southampton,Huddersfield Town,1.21,0.37,1,1,1.15,1.04,1.41,4.90,7.50,0,1
378,Tottenham Hotspur,Everton,2.00,1.05,2,2,1.23,1.80,2.10,3.50,3.55,0,1


In [8]:
# 팀의 정보를 하나의 column에 담았으니 쓸모 없어진 홈팀, 어웨이팀 득점 수, difference column은 삭제한다

df.drop(['home_team_goal_count', 'away_team_goal_count', 'difference'], axis=1, inplace=True)
df

,home_team_name,away_team_name,home_ppg,away_ppg,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,home_team_result
0,Manchester United,Leicester City,1.89,1.32,1.25,1.46,1.37,4.98,9.81,2
1,Newcastle United,Tottenham Hotspur,1.32,1.74,1.18,1.58,4.51,3.77,1.84,0
2,AFC Bournemouth,Cardiff City,1.53,0.74,1.50,0.89,2.03,3.51,3.96,2
3,Fulham,Crystal Palace,1.11,1.53,1.67,1.81,2.31,3.44,3.26,0
4,Huddersfield Town,Chelsea,0.47,1.58,0.90,1.17,7.47,4.27,1.51,0
...,...,...,...,...,...,...,...,...,...,...
375,Liverpool,Wolverhampton Wanderers,2.79,1.21,1.40,0.92,1.32,5.80,9.00,2
376,Manchester United,Cardiff City,1.89,0.74,2.67,1.31,1.28,6.20,10.00,0
377,Southampton,Huddersfield Town,1.21,0.37,1.15,1.04,1.41,4.90,7.50,1
378,Tottenham Hotspur,Everton,2.00,1.05,1.23,1.80,2.10,3.50,3.55,1


In [9]:
# 새롭게 추가할 data는 home, away team의 근 3년간 시즌에서의 최종 승점의 정보이다
# 현재 18/19 시즌이기 때문에 15/16, 16/17, 17/18 시즌의 성적을 알아보자
df['home_team_name'].value_counts()

Wolverhampton Wanderers    19
West Ham United            19
Crystal Palace             19
AFC Bournemouth            19
Watford                    19
Burnley                    19
Manchester United          19
Fulham                     19
Huddersfield Town          19
Leicester City             19
Tottenham Hotspur          19
Liverpool                  19
Chelsea                    19
Manchester City            19
Cardiff City               19
Everton                    19
Brighton & Hove Albion     19
Newcastle United           19
Arsenal                    19
Southampton                19
Name: home_team_name, dtype: int64

In [10]:
season_1516 = {'Brighton & Hove Albion': 89*0.5, 'Cardiff City': 68*0.5, 'AFC Bournemouth': 42, 'Southampton': 63, 'Huddersfield Town': 51*0.5, 'Wolverhampton Wanderers': 58*0.5, 'Chelsea': 50, 'Watford': 45, 'Tottenham Hotspur': 70, 'West Ham United': 62, 'Liverpool': 60, 'Manchester United': 66, 'Burnley': 93*0.5, 'Crystal Palace': 42, 'Fulham': 51*0.5, 'Leicester City': 81, 'Manchester City': 66, 'Newcastle United': 37, 'Everton': 47, 'Arsenal': 71}

season_1617 = {'Brighton & Hove Albion': 93*0.5, 'Cardiff City': 62*0.5, 'AFC Bournemouth': 46, 'Southampton': 46, 'Huddersfield Town': 81*0.5, 'Wolverhampton Wanderers': 58*0.5, 'Chelsea': 93, 'Watford': 40, 'Tottenham Hotspur': 86, 'West Ham United': 45, 'Liverpool': 76, 'Manchester United': 69, 'Burnley': 40, 'Crystal Palace': 41, 'Fulham': 80*0.5, 'Leicester City': 44, 'Manchester City': 78, 'Newcastle United': 94*0.5, 'Everton': 61, 'Arsenal': 75}

season_1718 = {'Brighton & Hove Albion': 40, 'Cardiff City': 90*0.5, 'AFC Bournemouth': 44, 'Southampton': 36, 'Huddersfield Town': 37, 'Wolverhampton Wanderers': 99*0.5, 'Chelsea': 70, 'Watford': 41, 'Tottenham Hotspur': 77, 'West Ham United': 42, 'Liverpool': 75, 'Manchester United': 81, 'Burnley': 54, 'Crystal Palace': 44, 'Fulham': 88*0.5, 'Leicester City': 47, 'Manchester City': 100, 'Newcastle United': 44, 'Everton': 49, 'Arsenal': 63}


for team, point in season_1617.items():
    point = point * 1.3
    season_1617[team] = point

for team, point in season_1718.items():
    point = point * 1.5
    season_1718[team] = point

data1516 = pd.DataFrame.from_dict([season_1516])

data1617 = pd.DataFrame.from_dict([season_1617])

data1718 = pd.DataFrame.from_dict([season_1718])

total_points = data1516 + data1617 + data1718
total_points

mean_of_points = total_points / 3
mean_of_points

,Brighton & Hove Albion,Cardiff City,AFC Bournemouth,Southampton,Huddersfield Town,Wolverhampton Wanderers,Chelsea,Watford,Tottenham Hotspur,West Ham United,Liverpool,Manchester United,Burnley,Crystal Palace,Fulham,Leicester City,Manchester City,Newcastle United,Everton,Arsenal
0,54.983333,47.266667,55.933333,58.933333,44.55,46.983333,91.966667,52.833333,99.1,61.166667,90.433333,92.4,59.833333,53.766667,47.833333,69.566667,105.8,54.7,66.6,87.666667


In [11]:
# 원래의 dataframe에 각 팀의 승점에 해당되는 점수를 넣는다.

for team in mean_of_points.columns:
    for index, teams in enumerate(df['home_team_name'].values):
        if str(teams) == str(team):     
            df['home_team_name'].values[index] = mean_of_points[team].values[0] # values들은 list 형태에 저장돼 있기에 index 0값을 불러 가져옴

for team in mean_of_points.columns:
    for index, teams in enumerate(df['away_team_name'].values):
        if str(teams) == str(team):     
            df['away_team_name'].values[index] = mean_of_points[team].values[0] # values들은 list 형태에 저장돼 있기에 index 0값을 불러 가져옴
df

,home_team_name,away_team_name,home_ppg,away_ppg,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,home_team_result
0,92.4,69.5667,1.89,1.32,1.25,1.46,1.37,4.98,9.81,2
1,54.7,99.1,1.32,1.74,1.18,1.58,4.51,3.77,1.84,0
2,55.9333,47.2667,1.53,0.74,1.50,0.89,2.03,3.51,3.96,2
3,47.8333,53.7667,1.11,1.53,1.67,1.81,2.31,3.44,3.26,0
4,44.55,91.9667,0.47,1.58,0.90,1.17,7.47,4.27,1.51,0
...,...,...,...,...,...,...,...,...,...,...
375,90.4333,46.9833,2.79,1.21,1.40,0.92,1.32,5.80,9.00,2
376,92.4,47.2667,1.89,0.74,2.67,1.31,1.28,6.20,10.00,0
377,58.9333,44.55,1.21,0.37,1.15,1.04,1.41,4.90,7.50,1
378,99.1,66.6,2.00,1.05,1.23,1.80,2.10,3.50,3.55,1


In [13]:
# team_name 에서 team_point 로 이름 변경

df = df.rename({'home_team_name':'home_team_point', 'away_team_name':'away_team_point'}, axis='columns')
df

,home_team_point,away_team_point,home_ppg,away_ppg,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,home_team_result
0,92.4,69.5667,1.89,1.32,1.25,1.46,1.37,4.98,9.81,2
1,54.7,99.1,1.32,1.74,1.18,1.58,4.51,3.77,1.84,0
2,55.9333,47.2667,1.53,0.74,1.50,0.89,2.03,3.51,3.96,2
3,47.8333,53.7667,1.11,1.53,1.67,1.81,2.31,3.44,3.26,0
4,44.55,91.9667,0.47,1.58,0.90,1.17,7.47,4.27,1.51,0
...,...,...,...,...,...,...,...,...,...,...
375,90.4333,46.9833,2.79,1.21,1.40,0.92,1.32,5.80,9.00,2
376,92.4,47.2667,1.89,0.74,2.67,1.31,1.28,6.20,10.00,0
377,58.9333,44.55,1.21,0.37,1.15,1.04,1.41,4.90,7.50,1
378,99.1,66.6,2.00,1.05,1.23,1.80,2.10,3.50,3.55,1


In [14]:
 # 이를 해주는 이유는 dataframe에 새로 추가해준 승점 정보를 숫자형으로 확실하게 바꿔주기 위함이다.
df['home_team_point'] = pd.to_numeric(df['home_team_point'])
df['away_team_point'] = pd.to_numeric(df['away_team_point'])
df

,home_team_point,away_team_point,home_ppg,away_ppg,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,home_team_result
0,92.400000,69.566667,1.89,1.32,1.25,1.46,1.37,4.98,9.81,2
1,54.700000,99.100000,1.32,1.74,1.18,1.58,4.51,3.77,1.84,0
2,55.933333,47.266667,1.53,0.74,1.50,0.89,2.03,3.51,3.96,2
3,47.833333,53.766667,1.11,1.53,1.67,1.81,2.31,3.44,3.26,0
4,44.550000,91.966667,0.47,1.58,0.90,1.17,7.47,4.27,1.51,0
...,...,...,...,...,...,...,...,...,...,...
375,90.433333,46.983333,2.79,1.21,1.40,0.92,1.32,5.80,9.00,2
376,92.400000,47.266667,1.89,0.74,2.67,1.31,1.28,6.20,10.00,0
377,58.933333,44.550000,1.21,0.37,1.15,1.04,1.41,4.90,7.50,1
378,99.100000,66.600000,2.00,1.05,1.23,1.80,2.10,3.50,3.55,1


In [15]:
from sklearn.preprocessing import MinMaxScaler

target = df['home_team_result']
features = df.drop('home_team_result', axis=1, inplace=False)

scaler = MinMaxScaler()
scaler.fit(features)
df_scaled = scaler.transform(features)
soccer_df_scaled = pd.DataFrame(data=df_scaled, columns=features.columns)
soccer_df_scaled

,home_team_point,away_team_point,home_ppg,away_ppg,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win
0,0.781224,0.408435,0.599156,0.514563,0.365385,0.583333,0.014333,0.161868,0.208744
1,0.165714,0.890612,0.358650,0.718447,0.342949,0.638889,0.164357,0.067704,0.017295
2,0.185850,0.044354,0.447257,0.233010,0.445513,0.319444,0.045867,0.047471,0.068220
3,0.053605,0.150476,0.270042,0.616505,0.500000,0.745370,0.059245,0.042023,0.051405
4,0.000000,0.774150,0.000000,0.640777,0.253205,0.449074,0.305781,0.106615,0.009368
...,...,...,...,...,...,...,...,...,...
375,0.749116,0.039728,0.978903,0.461165,0.413462,0.333333,0.011945,0.225681,0.189287
376,0.781224,0.044354,0.599156,0.233010,0.820513,0.513889,0.010033,0.256809,0.213308
377,0.234830,0.000000,0.312236,0.053398,0.333333,0.388889,0.016245,0.155642,0.153255
378,0.890612,0.360000,0.645570,0.383495,0.358974,0.740741,0.049212,0.046693,0.058371


### Machine Learning model for classification

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [74]:
X_train, X_test, y_train, y_test = train_test_split(soccer_df_scaled, target, test_size=0.2)

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

ab = AdaBoostClassifier()
ab.fit(X_train, y_train)
ab_pred = ab.predict(X_test)

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

classifiers = [dt_pred, rf_pred, lr_pred, ab_pred, knn_pred]

for i in classifiers:
    print(accuracy_score(y_test, i))

0.5526315789473685
0.5657894736842105
0.7105263157894737
0.5657894736842105
0.5394736842105263
